# DREAM Diffusion - High-Quality Face Generation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/dream-diffusion/blob/main/notebooks/dream_diffusion_colab.ipynb)
[![GitHub](https://img.shields.io/badge/GitHub-Repository-blue)](https://github.com/yourusername/dream-diffusion)

This notebook implements **DREAM (Diffusion Rectification and Estimation-Adaptive Models)** for face generation using the CelebA dataset.

## Key Features:
- 🚀 **DREAM Framework**: Advanced diffusion training with rectification loss
- 🛡️ **Crash Protection**: Auto-recovery training with checkpoint management
- ⚡ **Memory Optimized**: Efficient for consumer GPUs (RTX 3070+)
- 📊 **Comprehensive Evaluation**: FID, Inception Score, visual metrics
- 🔧 **Conservative Settings**: Stable hyperparameters for reproducible results

## Usage Instructions:
1. **Run cells in order** - Each cell is numbered and contains specific functionality
2. **Automatic dataset download** - CelebA will be downloaded automatically
3. **Crash protection** - Training will auto-resume from checkpoints
4. **Memory management** - Optimized for 8GB+ GPU memory

## Expected Results:
- **Training Time**: ~14-20 hours on Tesla V100/A100
- **FID Score**: Expected < 30 (lower is better)
- **Inception Score**: Expected > 2.5 (higher is better)
- **Sample Quality**: High-resolution 64×64 face generation

---
**Note**: This is a research implementation. For production use, please see the [GitHub repository](https://github.com/yourusername/dream-diffusion).

## 🚀 Setup and Environment Configuration

In [ ]:
# Cell 1: GPU Setup and Environment Check
!nvidia-smi
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Keep Alive Function for longer training sessions
def keep_alive():
    """Keeps Colab session active during training"""
    import IPython
    from datetime import datetime
    print(f"📡 Session active: {datetime.now().strftime('%H:%M:%S')}")
    return True

print("✅ GPU setup complete!")

In [ ]:
# Cell 2: Mount Google Drive and Setup Directories
from google.colab import drive
drive.mount('/content/drive')

import os
import glob

# Create project directories
project_dir = '/content/drive/MyDrive/dream_diffusion'
checkpoint_dir = f'{project_dir}/checkpoints'
output_dir = f'{project_dir}/outputs'
eval_dir = f'{project_dir}/evaluation'

os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(eval_dir, exist_ok=True)

# Check for existing checkpoints (crash recovery)
latest_checkpoint = os.path.join(checkpoint_dir, 'latest.pt')
if os.path.exists(latest_checkpoint):
    print("🔄 CRASH RECOVERY: Previous training found!")
    checkpoint_info = torch.load(latest_checkpoint, map_location='cpu', weights_only=False)
    print(f"📊 Last epoch: {checkpoint_info.get('epoch', 'Unknown')}")
    print(f"📊 Last loss: {checkpoint_info.get('loss', 'Unknown'):.4f}")
    print("⚠️  Training will resume automatically")
else:
    print("🆕 Fresh start - no previous training found")

print("✅ Directory setup complete!")
print(f"📁 Checkpoints: {checkpoint_dir}")
print(f"📁 Outputs: {output_dir}")
print(f"📁 Evaluation: {eval_dir}")

In [ ]:
# Cell 3: Install Required Packages
print("📦 Installing required packages...")
!pip install -q einops accelerate tensorboard
!pip install -q torch-fidelity clean-fid lpips scipy
!pip install -q gdown  # For dataset download

print("✅ All packages installed!")
print("📊 Evaluation tools ready: FID, IS, LPIPS")

In [ ]:
# Cell 4: Dataset Download and Verification
dataset_path = '/content/img_align_celeba'

if os.path.exists(dataset_path):
    num_images = len([f for f in os.listdir(dataset_path) if f.endswith('.jpg')])
    print(f"✅ Dataset already exists: {num_images:,} images")
else:
    print("📥 Downloading CelebA dataset...")
    !gdown --id 1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684 -O celeba.zip
    print("📂 Extracting dataset...")
    !unzip -q celeba.zip -d /content/
    !rm celeba.zip

    if os.path.exists(dataset_path):
        num_images = len([f for f in os.listdir(dataset_path) if f.endswith('.jpg')])
        print(f"✅ Dataset ready: {num_images:,} images")
    else:
        print("❌ Dataset download failed!")
        raise Exception("Please check your internet connection and try again")

# Check available disk space
!df -h /content
print("💾 Disk space check complete")

In [ ]:
# Cell 5: Session Keep-Alive (Anti-Disconnect)
from IPython.display import HTML, Javascript

# JavaScript code to keep session alive
js_code = """
// Auto-clicker for Colab (Prevents timeout)
function ClickConnect(){
    console.log("🔄 Keeping session alive...");
    var connectButton = document.querySelector("colab-connect-button");
    if (connectButton) {
        connectButton.click();
    }
}

// Run every 60 seconds
var keepAliveInterval = setInterval(ClickConnect, 60000);
console.log("🚀 Auto-clicker started - Session will stay alive!");

// To stop manually: clearInterval(keepAliveInterval)
"""

display(Javascript(js_code))

print("🚀 Auto-clicker started!")
print("📡 Session crash protection active")
print("⚠️  This will prevent Colab from disconnecting during training")
print("\n💡 To stop manually, run: clearInterval(keepAliveInterval) in browser console")